In [88]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor
from lightgbm import LGBMRegressor

In [89]:
windy_power = pd.read_csv("./dataset/1_windy_power_2024-01-01_2024-12-31.csv")
windy_weather1 = pd.read_csv("./dataset/2_sunny_weather_1_2024-01-01_2025-02-28.csv")
windy_weather2 = pd.read_csv("./dataset/2_sunny_weather_2_2024-01-01_2025-02-28.csv")
windy_weather3 = pd.read_csv("./dataset/2_sunny_weather_3_2024-01-01_2025-02-28.csv")

suny_power = pd.read_csv("./dataset/2_sunny_power_2024-01-01_2024-12-31.csv")
suny_weather1 = pd.read_csv("./dataset/2_sunny_weather_1_2024-01-01_2025-02-28.csv")
suny_weather2 = pd.read_csv("./dataset/2_sunny_weather_2_2024-01-01_2025-02-28.csv")
suny_weather3 = pd.read_csv("./dataset/2_sunny_weather_3_2024-01-01_2025-02-28.csv")

windy_weather1.columns = ["time"] + ["NWP_1_" + x for x in windy_weather1.columns[1:]]
windy_weather2.columns = ["time"] + ["NWP_2_" + x for x in windy_weather2.columns[1:]]
windy_weather3.columns = ["time"] + ["NWP_3_" + x for x in windy_weather3.columns[1:]]

suny_weather1.columns = ["time"] + ["NWP_1_" + x for x in suny_weather1.columns[1:]]
suny_weather2.columns = ["time"] + ["NWP_2_" + x for x in suny_weather2.columns[1:]]
suny_weather3.columns = ["time"] + ["NWP_3_" + x for x in suny_weather3.columns[1:]]

In [90]:
windy_power["time"] = pd.to_datetime(windy_power["time"])
windy_weather1["time"] = pd.to_datetime(windy_weather1["time"])
windy_weather2["time"] = pd.to_datetime(windy_weather2["time"])
windy_weather3["time"] = pd.to_datetime(windy_weather3["time"])

suny_power["time"] = pd.to_datetime(suny_power["time"])
suny_weather1["time"] = pd.to_datetime(suny_weather1["time"])
suny_weather2["time"] = pd.to_datetime(suny_weather2["time"])
suny_weather3["time"] = pd.to_datetime(suny_weather3["time"])

In [91]:
windy_train_test = windy_power.merge(windy_weather1, on="time", how="right").merge(windy_weather2, on="time", how="right").merge(windy_weather3, on="time", how="right")
windy_train_test['year'] = windy_train_test['time'].dt.year 
windy_train_test['month'] = windy_train_test['time'].dt.month 
windy_train_test['day'] = windy_train_test['time'].dt.day
windy_train_test['dayofweek_num'] = windy_train_test['time'].dt.dayofweek  
windy_train_test['dayofweek_name'] = windy_train_test['time'].dt.weekday

windy_train = windy_train_test[~windy_train_test["real_power"].isnull()]
windy_test = windy_train_test[windy_train_test["real_power"].isnull()].tail(5664)



suny_train_test = suny_power.merge(suny_weather1, on="time", how="right").merge(suny_weather3, on="time", how="right").merge(suny_weather3, on="time", how="right")
suny_train_test['year'] = suny_train_test['time'].dt.year 
suny_train_test['month'] = suny_train_test['time'].dt.month 
suny_train_test['day'] = suny_train_test['time'].dt.day
suny_train_test['dayofweek_num'] = suny_train_test['time'].dt.dayofweek  
suny_train_test['dayofweek_name'] = suny_train_test['time'].dt.weekday

suny_train = suny_train_test[~suny_train_test["real_power"].isnull()]
suny_test = suny_train_test[suny_train_test["real_power"].isnull()].tail(5664)

In [92]:
model = LGBMRegressor()
model.fit(windy_train.drop(["time", "real_power"], axis=1), windy_train["real_power"])
pred = model.predict(windy_test.drop(["time", "real_power"], axis=1))

ouput1 = pd.read_csv("output1.csv")
ouput1["predict_power"] = pred
ouput1.to_csv("output1.csv", index=None)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.020535 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3314
[LightGBM] [Info] Number of data points in the train set: 35015, number of used features: 22
[LightGBM] [Info] Start training from score 35.312050


In [93]:
model = LGBMRegressor()
model.fit(suny_train.drop(["time", "real_power"], axis=1), suny_train["real_power"])
pred = model.predict(suny_test.drop(["time", "real_power"], axis=1))

ouput2 = pd.read_csv("output2.csv")
ouput2["predict_power"] = pred
ouput2.to_csv("output2.csv", index=None)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001781 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3470
[LightGBM] [Info] Number of data points in the train set: 34694, number of used features: 22
[LightGBM] [Info] Start training from score 5.022930


In [94]:
!\rm output.zip

In [95]:
!zip output.zip output1.csv output2.csv

  adding: output1.csv (deflated 70%)
  adding: output2.csv (deflated 79%)
